In [13]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import os
import sqlite3
import csv


In [14]:
# Function to sanitize a string for SQLite table name
def sanitize_table_name(name):
    return ''.join(c for c in name if c.isalnum() or c == '_')

def create_database():
    # Connect to the database (or create it)
    conn = sqlite3.connect('Flask/new_database2.db')
    cursor = conn.cursor()
    conn.commit()

def create_state_table(state):
    # Connect to the database
    conn = sqlite3.connect('Flask/new_database2.db')
    cursor = conn.cursor()
    
    # Sanitize the state name to make it a valid table name
    state_table_name = sanitize_table_name(state)
    
    # Create a table for the state if it doesn't exist
    cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS {state_table_name} (
        ID TEXT NOT NULL,
        bed REAL NOT NULL,
        bath REAL NOT NULL,
        acre_lot REAL NOT NULL,
        city TEXT NOT NULL,
        state TEXT NOT NULL,
        zip_code TEXT NOT NULL,
        house_size REAL NOT NULL,
        price REAL NULL,
        sold_previously REAL NOT NULL,
        latitude REAL NOT NULL,
        longitude REAL NOT NULL
    )
    ''')
    
    conn.commit()
    conn.close()

def load_data_from_csv(csv_file, state):
    # Connect to the database
    conn = sqlite3.connect('Flask/new_database2.db')
    cursor = conn.cursor()
    
    # Open and read the CSV file
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row
        for row in reader:
            cursor.execute(f'INSERT INTO {sanitize_table_name(state)} ("ID","bed","bath","acre_lot", "city", "state", "zip_code", "house_size", "price", "sold_previously", "latitude","longitude") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
                           (row[0], float(row[1]), float(row[2]), float(row[3]), row[4], row[5], row[6], float(row[7]), float(row[8]), float(row[9]), float(row[10]), float(row[11])))
    
    conn.commit()
    conn.close()

if __name__ == "__main__":
    create_database()
    # List of CSV file names
    csv_directory = 'new_data'
    csv_files = os.listdir(csv_directory)
    
    for csv_file in csv_files:
        if csv_file.endswith(".csv"):
            state_name = os.path.splitext(csv_file)[0]  # Extract state name from the filename
            create_state_table(state_name)
            csv_file_path = os.path.join(csv_directory, csv_file)
            load_data_from_csv(csv_file_path, state_name)

In [15]:
def rename_tables(database_path):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    
    # Get the current table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_names = [table[0] for table in cursor.fetchall()]
    
    # Rename each table by removing the "_properties" suffix
    for old_name in table_names:
        new_name = old_name.replace("_properties", "").lower()
        if new_name != old_name:
            cursor.execute(f"ALTER TABLE {old_name} RENAME TO {new_name};")
    
    conn.commit()
    conn.close()

if __name__ == "__main__":
    database_path = 'Flask/new_database2.db'  # Update with your database file path
    rename_tables(database_path)
    print("Tables renamed successfully.")

OperationalError: there is already another table or index with this name: connecticut

In [ ]:
def get_table_names(database_path):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    
    # Query to get table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    
    table_names = [table[0] for table in cursor.fetchall()]
    
    conn.close()
    
    return table_names

if __name__ == "__main__":
    database_path = 'Flask/new_database2.db'  
    table_names = get_table_names(database_path)
    
    print("Table Names:")
    for table_name in table_names:
        print(table_name)

Table Names:
connecticut
maine
massachusetts
newhampshire
newjersey
newyork
pennsylvania
puertorico
rhodeisland
vermont


In [ ]:
engine = create_engine("sqlite:///combinedstates.sqlite")
conn = engine.connect()

In [ ]:
def read_data_from_database():
    # Connect to the database
    conn = sqlite3.connect('Flask/new_database2.db')
    cursor = conn.cursor()
    # Execute a SELECT query
    cursor.execute('SELECT "ID","bed","bath","acre_lot", "city", "state", "zip_code", "house_size", "price", "sold_previously", "latitude","longitude" FROM NewYork')
    # Fetch all results
    users = cursor.fetchall()
    # Close the connection
    conn.close()
    return users
if __name__ == "__main__":
    users_data = read_data_from_database()
    

In [ ]:
users_data[0][4]

'Claverack'

In [ ]:
conn.close()